In [1]:
import import_ipynb
from DataRead import *
from math import sqrt, log

g = 9.8065                 
NODATA = -9999.

CAMPBELL = 1
RESTRICTED_VG = 2
IPPISCH_VG = 3
VAN_GENUCHTEN = 4

CELL_CENT_FIN_VOL = 1
NEWTON_RAPHSON_MP = 2
NEWTON_RAPHSON_MFP = 3

LOGARITHMIC = 0
HARMONIC = 1
GEOMETRIC = 2

class Csoil:
    upperDepth = NODATA        
    lowerDepth = NODATA        
    Campbell_he = NODATA      
    Campbell_b = NODATA        
    CampbellMFP_he = NODATA   
    Campbell_b3 = NODATA 
    VG_alpha = NODATA         
    VG_n = NODATA             
    VG_m = NODATA
    VG_he = NODATA             
    VG_alpha_mod = NODATA      
    VG_n_mod = NODATA          
    VG_m_mod = NODATA          
    VG_Sc = NODATA             
    VG_thetaR = NODATA        
    Mualem_L = NODATA         
    thetaS = NODATA            
    Ks = NODATA     

def readSoil(soilFileName):
    mySoil = []
    A, isFileOk = readDataFile(soilFileName, 1, ',', False)
    if ((not isFileOk) or (len(A[0]) < 12)):
        print("error: wrong soil file.")
        return False, mySoil
    
    for i in range(len(A)):
        horizon = Csoil()
        horizon.upperDepth = A[i,0]
        horizon.lowerDepth = A[i,1]
        horizon.Campbell_he = A[i,2]
        horizon.Campbell_b = A[i,3]
        horizon.Campbell_n = 2.0 + (3.0 / horizon.Campbell_b)
        horizon.VG_he = A[i,4] ## Unit J/kg (m2/s2)
        horizon.VG_alpha = A[i,5]
        horizon.VG_n = A[i,6]
        horizon.VG_m =  1. - (1. / horizon.VG_n)
        horizon.VG_alpha_mod = A[i,7]
        horizon.VG_n_mod = A[i,8]
        horizon.VG_m_mod = 1. - (1. / horizon.VG_n_mod)
        horizon.VG_Sc = ((1.+ (horizon.VG_alpha_mod*abs(horizon.VG_he))**horizon.VG_n_mod)**(-horizon.VG_m_mod))
        horizon.VG_thetaR = A[i,9]
        horizon.thetaS = A[i,10]
        horizon.Ks = A[i,11]
        horizon.Mualem_L = 0.5
        horizon.CampbellMFP_he = horizon.Ks * horizon.Campbell_he / (1.0 - horizon.Campbell_n) 
        horizon.Campbell_b3 = (2.0 * horizon.Campbell_b + 3.0) / (horizon.Campbell_b + 3.0)
        mySoil.append(horizon)
    return True, mySoil
    
def thetaFromSe(Soil, Se):
    theta = (Se * (Soil.thetaS - Soil.VG_thetaR) + Soil.VG_thetaR)
    return(theta)    
    
def getHorizonIndex(Soil, depth):
    for index in range(len(Soil)):
        if ((depth >= Soil[index].upperDepth) 
        and (depth < Soil[index].lowerDepth)): 
            return (index)
    lastHorizon = len(Soil)-1
    if depth >= Soil[lastHorizon].lowerDepth:
        return lastHorizon
    else:
        return (-1)

def airEntryPotential(Soil): 
    return (Soil.VG_he)

def SeFromTheta(Soil, theta):
    if (theta >= Soil.thetaS): 
        return(1.)
    else:
        Se = (theta - Soil.VG_thetaR) / (Soil.thetaS - Soil.VG_thetaR)
    return (Se)
    
def hydraulicConductivityFromTheta(Soil, theta): 
    k = NODATA      
    Se = SeFromTheta(Soil, theta)
    num   = 1. - pow(1. - pow(Se * Soil.VG_Sc, 1./ Soil.VG_m_mod), Soil.VG_m_mod);
    denom = 1. - pow(1. - pow(Soil.VG_Sc, 1./ Soil.VG_m_mod), Soil.VG_m_mod);
    k = Soil.Ks * pow(Se, Soil.Mualem_L) * pow((num / denom), 2.)
    return (k)
    
def waterPotential(Soil, theta):
    psi = NODATA
    Se = SeFromTheta(Soil, theta)
    psi = -((1./Soil.VG_alpha_mod) * ((1./(Se*Soil.VG_Sc))**(1./Soil.VG_m_mod)-1.)**(1./Soil.VG_n_mod))
    return (psi)
    
def degreeOfSaturation(Soil, psi):
    if (psi >= 0.): return(1.)
    Se = NODATA
    if (psi >= Soil.VG_he): 
        Se = 1.
    else: 
        Se = (1./Soil.VG_Sc) * pow(1.+pow(Soil.VG_alpha_mod 
             * abs(psi), Soil.VG_n_mod), -Soil.VG_m_mod)
    return (Se) 

def thetaFromPsi(Soil, psi):
    Se = degreeOfSaturation(Soil, psi)
    theta = thetaFromSe(Soil, Se)
    return (theta)

def dTheta_dPsi(Soil, psi):
    airEntry = airEntryPotential(Soil)
    if (psi > airEntry): 
        return 0.0
    dSe_dpsi = Soil.VG_alpha_mod * Soil.VG_n_mod * (Soil.VG_m_mod * pow(1. +\
               pow(Soil.VG_alpha_mod * abs(psi), Soil.VG_n_mod), -(Soil.VG_m_mod + 1.))\
                                                    * pow(Soil.VG_alpha_mod * abs(psi), Soil.VG_n_mod - 1.))      
    dSe_dpsi *= (1. / Soil.VG_Sc)
    return dSe_dpsi * (Soil.thetaS - Soil.VG_thetaR)
    
def dTheta_dH(Soil, H0, H1, z): 
    psi0 = H0 + g*z
    psi1 = H1 + g*z
    if (abs(psi1-psi0) < 1E-5):
        return dTheta_dPsi(Soil, psi0)
    else:
        theta0 = thetaFromPsi(Soil, psi0)
        theta1 = thetaFromPsi(Soil, psi1)
        return (theta1 - theta0) / (psi1 - psi0)
    
def meanK(meanType, k1, k2):
    if (meanType == LOGARITHMIC):
        if (k1 != k2):
            k = (k1-k2) / log(k1/k2)
        else:
            k = k1
    elif (meanType == HARMONIC): 
        k = 2.0 / (1.0 / k1 + 1.0 / k2)
    elif (meanType == GEOMETRIC): 
        k = sqrt(k1 * k2)
    return k

importing Jupyter notebook from DataRead.ipynb


ModuleNotFoundError: No module named 'astropy'